## An introduction to Julia (RSE 2018)
#### by Valentin Churavy, JuliaLab, CSAIL, MIT

```
               _
   _       _ _(_)_     
  (_)     | (_) (_)    
   _ _   _| |_  __ _   
  | | | | | | |/ _` |
  | | |_| | | | (_| |
 _/ |\__'_|_|_|\__'_|
|__/                 
```



# Following along!
1. Use https://juliabox.org to get a cloud instance of Jupyter with Julia
2. Follow the instructions on https://julialang.org/downloads/
3. To use Jupyter install IJulia.jl https://github.com/JuliaLang/IJulia.jl
4. VSCode has great Julia support

# What is Julia?
*Walks like Python, talks like Lips, runs like Fortran*
- Modern high-level dynamic programming language
- Engineered with performance in mind
- n-based array access (but opinionated about 1-based)
- Julia Manifesto: https://julialang.org/blog/2012/02/why-we-created-julia
- Noteworthy differences: https://docs.julialang.org/en/latest/manual/noteworthy-differences/
- The Julia promise: Within 2x of C at the ease of Python

# Why Julia?

*Come for the speed, stay for the productivity*

- Elegant
- Explorable & Understandable
- No privileged types/code
- No need to switch languages
- Code that is close to the mathematics

# What is the 2 language problem?
You start out proto-typing in one language (high-level, dynamic),
but performance forces you to switch to a different one (low-level, static).


- For **convinience** use a scripting language (Python, R, Matlab, ...)
- but do all the **hard stuff** in a systems language (C, C++, Fortran)

Pragmatic for many applications, but has drawbacks
- aren't the **hard parts** exactly where you need an **easier** language
- creates a **social barrier** -- a wall between users and developers
- **"sandwich problem"** -- layering of system & user code is expensive
- **prohibits** full stack optimisations

# Julia for RSEs?
*Tearing down barriers of collaboration*

- Fostering collaboration
- Low-barrier from package user to package developer
- One codebase to rule them all
- Poster child: Celeste.jl
- Understandable and explorable performance

# Julia now!
- v"1.0.0" tagged and released August 8th, 2018
- stable foundation to build tools upon
- we promise less breakage
- comes with a new package manager that focuses on reproducibility
- JuliaCon >350 attendees, sold-out, all talks available and live-streamed.
- Excellent native GPU computing support

# The future!
- Improvements for parallel, heterogeous, and distributed computing (ask me later!)
- Composable machine learning
  - Model is a program, not data
  - Differentiable programming

# Resources
- Documentation: https://docs.julialang.org/en/stable/
- Forum: https://discourse.julialang.org/
- Issue tracker: https://github.com/JuliaLang/julia
- Join us on Slack: https://slackinvite.julialang.org/
- Pkgs: https://juliaobserver.com/ and https://pkg.julialang.org/pulse
- https://benlauwens.github.io/ThinkJulia.jl/latest/book.html
- https://www.youtube.com/channel/UC9IuUwwE2xdjQUT_LMLONoA

# Is Julia fast?
*Enough talk -- let's code*

$$
sum(a) = \sum_i^n a_i
$$


- This material began life as a wonderful [lecture by Steven Johnson at MIT](https://github.com/stevengj/18S096-iap17/blob/master/lecture1/Boxes-and-registers.ipynb).
- With apologies to the numerical computing folks, this is not the algorithm you should use!


In [1]:
# pick a large N to not measure call-overhead
data = rand(10_000_000);

In [2]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
pkg"precompile"

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %Precompiling project...


In [3]:
C_code =  """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

In [4]:
using Libdl
const Clib = tempname()   # make a temporary file

# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

In [5]:
function c_sum(X::Array{Float64})
    ccall(("c_sum", Clib), 
          Float64,
          (Csize_t, Ref{Float64}),
          length(X), X)
end

c_sum (generic function with 1 method)

In [6]:
using PyCall

# Get two python objects that represent data
# First a list and then a numpy array
# We do this to cut down conversion overhead
apy_list = PyCall.array2py(data, 1, 1)
apy_numpy = PyObject(data)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum");
# get the Numpy "sum" function:
numpy_sum = pyimport("numpy")["sum"];

In [7]:
py"""
def py_sum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

sum_py = py"py_sum";

In [8]:
function mysum(data)
  acc = zero(eltype(data))
  for x in data
      acc += x
  end
  return acc
end

mysum (generic function with 1 method)

In [9]:
@which sum(data)

sum(a::AbstractArray) in Base at reducedim.jl:645

In [10]:
using BenchmarkTools

suite = BenchmarkGroup()
suite["Julia handwritten"]       = @benchmarkable mysum($data)
suite["Julia builtin"]           = @benchmarkable sum($data)
suite["Simple C function"]       = @benchmarkable c_sum($data)
suite["Python builtin (list)"]   = @benchmarkable $pysum($apy_list)
suite["Python builtin (numpy)"]  = @benchmarkable $numpy_sum($apy_numpy)
suite["Python handwritten"]      = @benchmarkable $sum_py($apy_list)

# If a cache of tuned parameters already exists, use it, otherwise, tune and cache
# the benchmark parameters. Reusing cached parameters is faster and more reliable
# than re-tuning `suite` every time the file is included.
paramspath = joinpath(@__DIR__, "sum_bench.json")

if isfile(paramspath)
    loadparams!(suite, BenchmarkTools.load(paramspath)[1], :evals);
else
    tune!(suite)
    BenchmarkTools.save(paramspath, params(suite));
end

6-element BenchmarkTools.BenchmarkGroup:
  tags: []
  "Julia builtin" => Benchmark(evals=1, seconds=5.0, samples=10000)
  "Julia handwritten" => Benchmark(evals=1, seconds=5.0, samples=10000)
  "Simple C function" => Benchmark(evals=1, seconds=5.0, samples=10000)
  "Python builtin (list)" => Benchmark(evals=1, seconds=5.0, samples=10000)
  "Python builtin (numpy)" => Benchmark(evals=1, seconds=5.0, samples=10000)
  "Python handwritten" => Benchmark(evals=1, seconds=5.0, samples=10000)

In [11]:
results = run(suite)

6-element BenchmarkTools.BenchmarkGroup:
  tags: []
  "Julia builtin" => Trial(4.354 ms)
  "Julia handwritten" => Trial(10.301 ms)
  "Simple C function" => Trial(10.393 ms)
  "Python builtin (list)" => Trial(38.724 ms)
  "Python builtin (numpy)" => Trial(4.235 ms)
  "Python handwritten" => Trial(224.656 ms)

In [12]:
for (name, trial) in sort(collect(results), by=x->time(x[2]))
    t = time(trial) / 1e6
    println(rpad(name, 25, "."), lpad(string(round(t, digits=2), " ms"), 20, "."))
end

Python builtin (numpy)................4.24 ms
Julia builtin.........................4.35 ms
Julia handwritten.....................10.3 ms
Simple C function....................10.39 ms
Python builtin (list)................38.72 ms
Python handwritten..................224.66 ms


# In conclusion:
The point is not that Julia has the fasted sum, the point is that a simple Julia implementation (that looks and feels like Python) is as fast as simple C, and that well optimised Julia code can be even faster.

We can improve performance by giving the compiler more information `@inbounds` and `@simd`

# Improving performance

In [13]:
function mysum2(data)
    acc = zero(eltype(data))
    @simd for x in data
        acc += x
    end
    acc
end

mysum2 (generic function with 1 method)

In [14]:
@benchmark mysum2($data)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.518 ms (0.00% GC)
  median time:      5.235 ms (0.00% GC)
  mean time:        5.550 ms (0.00% GC)
  maximum time:     15.395 ms (0.00% GC)
  --------------
  samples:          899
  evals/sample:     1

# Aside: Which method

In [15]:
@which sum(data)

sum(a::AbstractArray) in Base at reducedim.jl:645

In [16]:
@which mapreduce(identity, Base.add_sum, data)

mapreduce(f, op, A::AbstractArray) in Base at reducedim.jl:301

In [17]:
@which Base._mapreduce(identity, Base.add_sum, IndexStyle(data), data)

_mapreduce(f, op, ::IndexLinear, A::AbstractArray{T,N} where N) where T in Base at reduce.jl:300

In [18]:
inds = LinearIndices(data)
@which Base.mapreduce_impl(identity, Base.add_sum, data, first(inds), last(inds))

mapreduce_impl(f, op, A::AbstractArray, ifirst::Integer, ilast::Integer) in Base at reduce.jl:176

In [19]:
@show blksize = Base.pairwise_blocksize(identity, Base.add_sum)
@which Base.mapreduce_impl(identity, Base.add_sum, data, first(inds), last(inds),blksize)

blksize = Base.pairwise_blocksize(identity, Base.add_sum) = 1024


mapreduce_impl(f, op, A::AbstractArray, ifirst::Integer, ilast::Integer, blksize::Int64) in Base at reduce.jl:154

# Basic Julia
- We have seen how to define a function
- We gotten a hint of FFI support
- Important syntax/concepts
    - multiple dispatch
    - macros
    - broadcasting

# Multiple dispatch in a nutshell
*think templates/method overloading but more powerful*

In [20]:
f(x) = "x is Any"
f(x::Number) = "x is a Number"
f(x::Float32) = "x is a Float32"

f (generic function with 3 methods)

In [21]:
methods(f)

# 3 methods for generic function "f":
[1] f(x::Float32) in Main at In[20]:3
[2] f(x::Number) in Main at In[20]:2
[3] f(x) in Main at In[20]:1

In [22]:
f(x::Float64, y::Float64) = 2x + y

f (generic function with 4 methods)

In [23]:
f(1.0, 2.0)

4.0

In [24]:
f(1.0, 1)

LoadError: MethodError: no method matching f(::Float64, ::Int64)
Closest candidates are:
  f(::Float64, !Matched::Float64) at In[22]:1
  f(::Number) at In[20]:2
  f(::Any) at In[20]:1

In [25]:
f(x::Number, y::Number) = 2x + y

f (generic function with 5 methods)

In [26]:
f(1.0, 1)

3.0

In [27]:
methods(+)

# 167 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[2] +(x::Bool, y::Bool) in Base at bool.jl:104
[3] +(x::Bool) in Base at bool.jl:101
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[5] +(x::Bool, z::Complex) in Base at complex.jl:284
[6] +(a::Float16, b::Float16) in Base at float.jl:392
[7] +(x::Float32, y::Float32) in Base at float.jl:394
[8] +(x::Float64, y::Float64) in Base at float.jl:395
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:292
[11] +(::Missing, ::Missing) in Base at missing.jl:92
[12] +(::Missing) in Base at missing.jl:79
[13] +(::Missing, ::Number) in Base at missing.jl:93
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:353
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:443
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:442
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:441
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:412
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:449
[21] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:455
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:503
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:496
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:490
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:349
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:318
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:325
[28] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:333
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:341
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Dates/src/periods.jl:349
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Dates/src/periods.jl:347
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Dates/src/periods.jl:385
[33] +(x::Dates.Date, y::Dates.Day) in Dates at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:78
[34] +(x::Dates.Date, y::Dates.Week) in Dates at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:76
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:59
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:32
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:20
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:24
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:82
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:52
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:28
[42] +(x::Dates.DateTime, y::Dates.Period) in Dates at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:80
[43] +(B::BitArray{2}, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/LinearAlgebra/src/uniformscaling.jl:88
[44] +(a::Pkg.Resolve.VersionWeights.VersionWeight, b::Pkg.Resolve.VersionWeights.VersionWeight) in Pkg.Resol

## Macros

Macros are syntax transformations and are a form of metaprogramming.
In Julia they always start with `@`.

In [28]:
@time sum(rand(10))

  0.008905 seconds (352 allocations: 19.016 KiB)


3.4258079903673506

In [29]:
@macroexpand @time sum(rand(10))

quote
    #= util.jl:154 =#
    local #18#stats = (Base.gc_num)()
    #= util.jl:155 =#
    local #20#elapsedtime = (Base.time_ns)()
    #= util.jl:156 =#
    local #19#val = sum(rand(10))
    #= util.jl:157 =#
    #20#elapsedtime = (Base.time_ns)() - #20#elapsedtime
    #= util.jl:158 =#
    local #21#diff = (Base.GC_Diff)((Base.gc_num)(), #18#stats)
    #= util.jl:159 =#
    (Base.time_print)(#20#elapsedtime, (#21#diff).allocd, (#21#diff).total_time, (Base.gc_alloc_count)(#21#diff))
    #= util.jl:161 =#
    (Base.println)()
    #= util.jl:162 =#
    #19#val
end

## Broadcast and dotfusion
If you are used to matlab than the following will look familiar.
```julia
A = rand(10)
B = rand(10)
A .= sin.(A) .+ B ./ A
```
Dots signify elementwise operations and we can fuse together these elementwise operations which will turn the above line to:

```julia
broadcast!((x, y, z) -> sin(x) + y / z, A, A, B, A)
```

This is similar to Matlab with a key difference being that it will work for user defined functions as well. Learn more at https://julialang.org/blog/2017/01/moredots

# Understandable performance

## A note on benchmarking
*Premature optimization is the root of all evil* & *If you don't measure you won't improve*

### Tools
1. BenchmarkTools.jl https://github.com/JuliaCI/BenchmarkTools.jl
2. Profiler https://docs.julialang.org/en/latest/manual/profile/
3. ProfileView.jl https://github.com/timholy/ProfileView.jl
4. VTunes/Perf/OProfile https://docs.julialang.org/en/latest/manual/profile/#External-Profiling-1

## BenchmarkTools.jl
Solid package that tries to eliminate common pitfalls in performance measurment.
- `@benchmark` macro that will repeatedly evaluate your code to gain enough samples
- Caveat: You probably want to escape `$` your input data

In [30]:
@benchmark sum($data)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.390 ms (0.00% GC)
  median time:      5.117 ms (0.00% GC)
  mean time:        5.439 ms (0.00% GC)
  maximum time:     16.367 ms (0.00% GC)
  --------------
  samples:          918
  evals/sample:     1

![Compiler](compiler.png)

![Compiler Stages](compiler-stages.png)

## Figuring out what is happening
The stages of the compiler
- `@code_lowered`
- `@code_typed` & `@code_warntype`
- `@code_llvm`
- `@code_native`

Where is a function defined
`@which` & `@edit`

# Let's revisit our example from earlier!

In [31]:
function mysum3(data)
  acc = 0
  for x in data
      acc += x
  end
  return acc
end

mysum3 (generic function with 1 method)

In [32]:
@code_lowered mysum3(zeros(3))

CodeInfo(
│2 1 ─       acc = 0
│3 │   %2  = data
│  │         #temp# = (Base.iterate)(%2)
│  │   %4  = #temp# === nothing
│  │   %5  = (Base.not_int)(%4)
│  └──       goto #4 if not %5
│  2 ┄ %7  = #temp#
│  │         x = (Core.getfield)(%7, 1)
│  │   %9  = (Core.getfield)(%7, 2)
│4 │         acc = acc + x
│  │         #temp# = (Base.iterate)(%2, %9)
│  │   %12 = #temp# === nothing
│  │   %13 = (Base.not_int)(%12)
│  └──       goto #4 if not %13
│  3 ─       goto #2
│6 4 ─       return acc
)

In [33]:
@code_typed optimize=false mysum3(zeros(3))

CodeInfo(
│2 1 ─       (acc = 0)::Const(0, false)
│3 │   %2  = data::Array{Float64,1}
│  │         (#temp# = (Base.iterate)(%2))::Union{Nothing, Tuple{Float64,Int64}}
│  │   %4  = (#temp# === nothing)::Bool
│  │   %5  = (Base.not_int)(%4)::Bool
│  └──       goto #4 if not %5
│  2 ┄ %7  = #temp#::Tuple{Float64,Int64}::Tuple{Float64,Int64}
│  │         (x = (Core.getfield)(%7, 1))::Float64
│  │   %9  = (Core.getfield)(%7, 2)::Int64
│4 │         (acc = acc + x)::Float64
│  │         (#temp# = (Base.iterate)(%2, %9))::Union{Nothing, Tuple{Float64,Int64}}
│  │   %12 = (#temp# === nothing)::Bool
│  │   %13 = (Base.not_int)(%12)::Bool
│  └──       goto #4 if not %13
│  3 ─       goto #2
│6 4 ─       return acc
) => Union{Float64, Int64}

In [34]:
@code_typed optimize=true mysum(zeros(3))

CodeInfo(
│╻╷╷  iterate3 1 ── %1  = (Base.arraylen)(data)::Int64
││╻╷   iterate  │    %2  = (Base.sle_int)(0, %1)::Bool
│││╻    <  │    %3  = (Base.bitcast)(UInt64, %1)::UInt64
││││╻    <  │    %4  = (Base.ult_int)(0x0000000000000000, %3)::Bool
││││╻    &  │    %5  = (Base.and_int)(%2, %4)::Bool
│││    └───       goto #3 if not %5
│││╻    getindex  2 ── %7  = (Base.arrayref)(false, data, 1)::Float64
│││    └───       goto #4
│││    3 ──       goto #4
││     4 ┄─ %10 = φ (#2 => false, #3 => true)::Bool
││     │    %11 = φ (#2 => %7)::Float64
││     │    %12 = φ (#2 => 2)::Int64
││     └───       goto #5
│      5 ── %14 = (Base.not_int)(%10)::Bool
│      └───       goto #11 if not %14
│      6 ┄─ %16 = φ (#5 => 0.0, #10 => %19)::Float64
│      │    %17 = φ (#5 => %11, #10 => %33)::Float64
│      │    %18 = φ (#5 => %12, #10 => %34)::Int64
│╻    +4 │    %19 = (Base.add_float)(%16, %17)::Float64
││╻    rem  │    %20 = (Base.bitcast)(UInt64, %18)::UInt64
││╻    -  │    %21 = π (0x0000000000

In [35]:
@code_warntype mysum3(zeros(3))

Body::Union{Float64, Int64}
│╻╷╷   iterate3 1 ── %1  = (Base.arraylen)(data)::Int64
││╻╷    iterate  │    %2  = (Base.sle_int)(0, %1)::Bool
│││╻     <  │    %3  = (Base.bitcast)(UInt64, %1)::UInt64
││││╻     <  │    %4  = (Base.ult_int)(0x0000000000000000, %3)::Bool
││││╻     &  │    %5  = (Base.and_int)(%2, %4)::Bool
│││     └───       goto #3 if not %5
│││╻     getindex  2 ── %7  = (Base.arrayref)(false, data, 1)::Float64
│││     └───       goto #4
│││     3 ──       goto #4
││      4 ┄─ %10 = φ (#2 => false, #3 => true)::Bool
││      │    %11 = φ (#2 => %7)::Float64
││      │    %12 = φ (#2 => 2)::Int64
││      └───       goto #5
│       5 ── %14 = (Base.not_int)(%10)::Bool
│       └───       goto #16 if not %14
│       6 ┄─ %16 = φ (#5 => 0, #15 => %32)::Union{Float64, Int64}
│       │    %17 = φ (#5 => %11, #15 => %46)::Float64
│       │    %18 = φ (#5 => %12, #15 => %47)::Int64
│     4 │    %19 = (isa)(%16, Float64)::Bool
│       └───       goto #8 if not %19
│       7 ── %21 = π

# And even lower! 

```
@code_llvm optimize=false mysum(data)

@code_llvm optimize=true mysum(data)

@code_native mysum(data)
```

# From performance to generic code
- Up until now I have been heavily focused on performance
- Mostly because I am a low-level person and this excites me!
- Performance was the reason why I came to Julia, but I stayed because of the features
- Let's talk about composable and generic code.

# AD in 5min
- In Julia there is no notion of privileged code
- User code can be as fast as code in the base language
- This enables powerful (and fast) high-level abstractions
- Automatic-Differentiation is a technique that powers all of modern ML

# Autodiff:  <br> Calculus  from another angle 
(and the special role played by Julia's multiple dispatch and compiler technology)

(based on a lecture given by Alan Edelman, 2017)


the first time I heard about automatic differentiation, it was easy for me to imagine what it was.  I was wrong.  In my head, I thought it was straightforward symbolic differentiation applied to code.  I kind of imagined it was like executing Mathematica or Maple, or even just automatically doing what I learned to do in my calculus class. 
  
![symbolic](http://www2.bc.cc.ca.us/resperic/math6a/lectures/ch5/1/IntegralTable.gif)


  .... and anyway if it was not that, then it must be finite differences, like one learns in a numerical computing class.
![finite differences](http://image.mathcaptain.com/cms/images/122/Diff%202.png)

## Babylonian sqrt

I would like to use a simple example, computation of sqrt(x), where for me how autodiff works came as both a mathematical surprise, and a computing wonder.  The example is  the Babylonian algorithm, known to man for millenia, to compute sqrt(x):  


 > Repeat $ t \leftarrow  (t+x/t) / 2 $ until $t$ converges to $\sqrt{x}$.
 
 Each iteration has one add and two divides. For illustration purposes, 10 iterations suffice.

In [36]:
function Babylonian(x, N = 10) 
    t = (1 + x) / 2
    for i in 2:N
        t = (t + x/t) / 2
    end    
    t
end  

Babylonian (generic function with 2 methods)

In [37]:
Babylonian(100)

10.0

Check that it works:

In [38]:
x=2; Babylonian(x),√x  # Type \sqrt+<tab> to get the symbol

(1.414213562373095, 1.4142135623730951)

and now the derivative (almost by magic)

In [39]:
struct Dual <: Number
    x::Float64
    dx::Float64
end

Sum Rule: (x+y)' = x' + y' <br>
Quotient Rule: (x/y)' = (yx'-xy') / y^2

In [40]:
import Base: +, -, *, /, convert, promote_rule
+(x::Dual, y::Dual) = Dual(x.x + y.x, x.dx + y.dx)
-(x::Dual, y::Dual) = Dual(x.x - y.x, x.dx - y.dx)
*(x::Dual, y::Dual) = Dual(x.x * y.x, x.dx * y.x + x.x*y.dx)
/(x::Dual, y::Dual) = Dual(x.x / y.x, (y.x*x.dx - x.x*y.dx)/y.x^2)
convert(::Type{Dual}, x::Real) = Dual(x, zero(x))
promote_rule(::Type{Dual}, ::Type{<:Number}) = Dual

promote_rule (generic function with 134 methods)

In [41]:
x=25.
Babylonian(Dual(x,1)), (√x,.5/√x)

(Dual(5.0, 0.1), (5.0, 0.1))

# Aside:
- If you want to do AD in Julia use ForwardDiff.jl instead of rolling your own
- We can also get a symbolic derivative (see SymPy.jl)

# What just happened?

In [42]:
function dBabylonian(x, N = 10) 
    t = (1+x)/2
    t′ = 1/2
    for i = 2:N;  
        t = (t+x/t)/2; 
        t′= (t′+(t-x*t′)/t^2)/2; 
    end    
    t′
end  

dBabylonian (generic function with 2 methods)

In [43]:
x = 2; dBabylonian(x), .5/√x

(0.35355339059327373, 0.35355339059327373)

In [44]:
Babylonian(Dual(x,1))

Dual(1.414213562373095, 0.35355339059327373)

# Aside 2:
There is actually a performance bug here...,
LLVM SLP vectoriser is to aggressive and over-optimises our `Babylonian`.

In [45]:
@benchmark Babylonian(25.0)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     12.792 ns (0.00% GC)
  median time:      14.221 ns (0.00% GC)
  mean time:        19.775 ns (0.00% GC)
  maximum time:     144.916 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     998

In [46]:
@benchmark Babylonian(Dual(25.0, 1.0))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     72.305 ns (0.00% GC)
  median time:      79.464 ns (0.00% GC)
  mean time:        101.681 ns (0.00% GC)
  maximum time:     1.449 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     971

In [47]:
@benchmark dBabylonian(25.0)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     34.924 ns (0.00% GC)
  median time:      37.538 ns (0.00% GC)
  mean time:        44.416 ns (0.00% GC)
  maximum time:     543.122 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     993

## Dual Number Notation

Instead of D(a,b) we can write a + b ϵ, where ϵ satisfies ϵ^2=0.  (Some people like to recall imaginary numbers where an i is introduced with i^2=-1.) 

Others like to think of how engineers just drop the O(ϵ^2) terms.

The four rules are

$ (a+b\epsilon) \pm (c+d\epsilon) = (a+c) \pm (b+d)\epsilon$

$ (a+b\epsilon) * (c+d\epsilon) = (ac) + (bc+ad)\epsilon$

$ (a+b\epsilon) / (c+d\epsilon) = (a/c) + (bc-ad)/d^2 \epsilon $


In [48]:
Base.show(io::IO,x::Dual) = print(io,x.x," + ",x.dx," ϵ")

In [49]:
const ϵ = Dual(0, 1)

0.0 + 1.0 ϵ

In [50]:
(1+ϵ)*(3+ϵ)

3.0 + 4.0 ϵ

In [51]:
1/(1+ϵ)  # Exact power series:  1-ϵ+ϵ²-ϵ³-...

1.0 + -1.0 ϵ

In [52]:
(1+ϵ)^10 ## Note this just works (we didn't train powers)!!

1.0 + 10.0 ϵ

# Distributed and GPU computing
Before you think about going to the GPU or distributed make sure that your single-core performance is stellar!

Secret Weapons:
- [StructOfArrays.jl](https://github.com/simonster/StructsOfArrays.jl)
- [StaticArrays.jl](https://github.com/JuliaArrays/StaticArrays.jl)

Julia has support for parallelism in the standard library and through packages:
- Distributed
- [DistributedArrays.jl](https://github.com/JuliaParallel/DistributedArrays.jl)
- [MPI.jl](https://github.com/JuliaParallel/MPI.jl)
- ...

In [53]:
using Distributed

In [54]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

we now have one master process and four worker processes

In [55]:
nprocs()

5

In [56]:
nworkers()

4

In [57]:
workers()

4-element Array{Int64,1}:
 2
 3
 4
 5

In [79]:
f_id = remotecall(myid, 2)

Future(2, 1, 67, nothing)

In [80]:
wait(f_id)

Future(2, 1, 67, nothing)

In [81]:
# Note: `fetch` will `wait` on the value being available
fetch(f_id)

2

`wait` and `fetch` require multiple messages to be send over the network. For efficiency two variants are provided.
- `remotecall_wait` = `wait(remotecall())`
- `remotecall_fetch` = `fetch(remotecall())`

In [61]:
remotecall_wait(x->2x, 4, 2) # Closures get serialized

Future(4, 1, 8, nothing)

In [62]:
g(x) = 2x
remotecall_fetch(g, 3, 2)

LoadError: On worker 3:
UndefVarError: #g not defined
deserialize_datatype at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Serialization/src/Serialization.jl:1051
handle_deserialize at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Serialization/src/Serialization.jl:743
deserialize at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Serialization/src/Serialization.jl:703
handle_deserialize at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Serialization/src/Serialization.jl:750
deserialize_msg at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Serialization/src/Serialization.jl:703
#invokelatest#1 at ./essentials.jl:686 [inlined]
invokelatest at ./essentials.jl:685 [inlined]
message_handler_loop at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Distributed/src/process_messages.jl:160
process_tcp_streams at /home/vchuravy/src/julia/usr/share/julia/stdlib/v1.0/Distributed/src/process_messages.jl:117
#105 at ./task.jl:259

In [63]:
@everywhere h(x) = 2x
remotecall_fetch(h, 3, 2)

4

# An excursion: Tasks

In [64]:
@sync begin
    @async (sleep(3); println("Today is reverse day!"))
    @async (sleep(2); println(" workshop!"))
    @async print("Hello")
end

Hello workshop!
Today is reverse day!


Task (done) @0x00007fa9d13539d0

In [65]:
@sync begin
    for wid in workers()
        @async remotecall_wait(println, wid, "Hello from worker ", string(wid))
    end
end

      From worker 2:	Hello from worker 2
      From worker 3:	Hello from worker 3
      From worker 4:	Hello from worker 4
      From worker 5:	Hello from worker 5


Use `@verywhere` to execute a top-level block on each process

In [82]:
@everywhere begin
    using Pkg
    Pkg.activate(@__DIR__)
end

In [68]:
# Define on all processes the variable bar
@everywhere bar = 1

In [69]:
foo =  10
@everywhere baz = $foo

In [83]:
@everywhere using DistributedArrays

In [87]:
# Distribute local array across processes
dA = distribute(ones(64, 64))

64×64 DArray{Float64,2,Array{Float64,2}}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1

In [102]:
# Convert a distributedArray to a local one
convert(Array, dA);

In [89]:
# access the process local part of the DArray
localpart(dA)
dA[:L]

0×0 Array{Float64,2}

In [94]:
# Which indexes are local?
localindices(dA)

(1:0, 1:0)

In [95]:
# Construct a distributed array
dA = dfill(0.0, 500, 500)
remotecall_fetch(()->size(localpart(dA)), 2)

(250, 250)

In [98]:
remotecall_fetch(localindices, 3, dA)

(251:500, 1:250)

In [100]:
# Darray(init, dims, [procs, dist])
dA = dfill(1.0, (128, 128), workers(), [1, nworkers()]);
remotecall_fetch(()->size(localpart(dA)), 2)

(128, 32)

In [101]:
function mymap!(f::F, dest::DArray, src::DArray) where F
    @sync for p in procs(dest)
        @async remotecall_wait(p) do
            map!(f, localpart(dest), src[localindexes(dest)...])
            nothing
        end
    end
    return dest
end

mymap! (generic function with 1 method)